- [ ] wie viel wir updaten 

- [ ] von was wir updaten 

- [ ] function um artikel: welcher artikel wurde um wie viel geupdated 

In [ ]:
!pip install flask==2.3.3
from flask import Flask, request, jsonify
import pandas as pd

app = Flask(__name__)

# Load inventory
def load_inventory(file_path):
    """Load inventory from a CSV file."""
    try:
        return pd.read_csv(file_path, sep=',')
    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{file_path}' does not exist.")
    except Exception as e:
        raise Exception(f"An error occurred while loading the inventory: {e}")

# Save inventory
def save_inventory(file_path, inventory):
    """Save inventory to a CSV file."""
    try:
        inventory.to_csv(file_path, index=False)
    except Exception as e:
        raise Exception(f"An error occurred while saving the inventory: {e}")

# Update inventory
def update_inventory(file_path, Name, quantity, operation):
    """
    Update the inventory for a specific product.

    Args:
        file_path (str): Path to the inventory file.
        Name (str): The ID of the product to update.
        quantity (int): The amount to add or remove.
        operation (str): The operation, either 'add' or 'remove'.

    Returns:
        pd.DataFrame: The updated inventory.
    """
    # Load the inventory
    inventory = load_inventory(file_path)
    
    # Check if the product exists
    print(f"before get inside------{inventory}")
    if Name in inventory['Name'].values:
        print("got inside")
        # Update quantity based on operation
        if operation == 'add':
            inventory.loc[inventory['Name'] == Name, 'quantity'] += quantity
        elif operation == 'remove':
            inventory.loc[inventory['Name'] == Name, 'quantity'] -= quantity
            # Ensure quantity doesn't drop below zero
            inventory.loc[inventory['Name'] == Name, 'quantity'] = \
                inventory.loc[inventory['Name'] == Name, 'quantity'].clip(lower=0)
        else:
            raise ValueError(f"Invalid operation '{operation}'. Supported operations: 'add', 'remove'.")
    else:
        # Raise an exception if the product ID is not found
        raise KeyError(f"Product Name '{Name}' not found in the inventory.")

    # Save the updated inventory
    save_inventory(file_path, inventory)
    return inventory


# Flask Routes
@app.route('/update_inventory', methods=['POST'])
def api_update_inventory():
    """API to update inventory."""
    try:
        data = request.json
        file_path = data['file_path']
        Name = data['Name']
        quantity = data['quantity']
        operation = data['operation']

        updated_inventory = update_inventory(file_path, Name, quantity, operation)
        return jsonify(updated_inventory.to_dict(orient='records')), 200
    except KeyError as e:
        return jsonify({'error': f'Missing key: {str(e)}'}), 400
    except ValueError as e:
        return jsonify({'error': str(e)}), 400
    except Exception as e:
        return jsonify({'error': str(e)}), 500


# Run the Flask app
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 34.1 MB/s eta 0:00:00
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.2.2
    Not uninstalling werkzeug at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'Werkzeug'. No files were found to uninstall.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
 * Serving Flask app '__main__'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://172.3.31.216:8080
Press CTRL+C to 

In [1]:
url -X POST   https://945152e0-66a8-4b9c-b068-364219c8e551.deepnoteproject.com/update_inventory   -H "Content-Type: application/json"   -d '{
        "file_path": "narcotics_bs.csv",
        "Name": "Methadon",
        "quantity": 50,
        "operation": "remove"
      }'


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=945152e0-66a8-4b9c-b068-364219c8e551' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>